<a href="https://colab.research.google.com/github/heinohen/tko_7095_i2hlt/blob/main/week5_ex_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install datasets more-itertools

In [2]:
import datasets
import sklearn.feature_extraction

In [3]:
data = datasets.load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
cvec = sklearn.feature_extraction.text.CountVectorizer(lowercase = False, stop_words = None, token_pattern=r"(?u)\b\w+\b" )
analyzer = cvec.build_analyzer()
analyzer('I have a dog at home, it likes to shred newspapers.')

['I',
 'have',
 'a',
 'dog',
 'at',
 'home',
 'it',
 'likes',
 'to',
 'shred',
 'newspapers']

## TASK A

In [5]:
# Tokenize the IMDB dataset

def tokenize(ex) -> dict:
  return {"tokenized":analyzer(ex["text"])}

data = data.map(tokenize, num_proc = 4)

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
from collections import Counter
from more_itertools import sliding_window
import tqdm

def generate_ngrams(dset, n ):
  for ex in tqdm.tqdm(dset):
    tokens = ["<bos>"] * (n-1) + ex["tokenized"]+["<eos>"]
    for ngram in sliding_window(tokens,n):
      yield ngram

## TASK B

In [7]:
# Concat all invid datasets (train, test, unlabelled) in IMDB
# The "master" dataset is a dict of thses, so dset.values() has the datasets of the individual sections
combined_dataset = datasets.concatenate_datasets(list(data.values()))

In [8]:
ngrams={}
for ngram in generate_ngrams(combined_dataset,4):
    prefix=ngram[:-1]
    word=ngram[-1]
    d=ngrams.setdefault(prefix,{})
    d[word]=d.get(word,0)+1

100%|██████████| 100000/100000 [01:30<00:00, 1110.89it/s]


*** TRIED THIS WITH MANY DIFFERENT COMBINATIONS ***

* convert to string and store with freq
* store as a tuple with freq
* tried storing as individiual words with freqs (i know... not n-gram)

Could not get it to work, had to resort looking at model solution...

But why do i need the prefix ?

## TASK C

In [9]:
import numpy as np

def softmax(x):
  return np.exp(x) / sum(np.exp(x))

def sample_from(counts, temperature=1.0):
  """
  counts: List of counts that form the distribution
  temperature: The "how wild the generation should be" parameter, numbers close to 0 are very conservative,
  numbers close or above 1 lead to quite wild generations
  """

  counts_array = np.array(counts)

  # Make thse sum up to 1
  counts_array_norm = counts_array / counts_array.sum()

  # Divide by temperature, that is what the algorithm does
  counts_array_norm /= temperature

  # Renormalize into a distribution using the softmax function, that is what the algorithm does
  final_distribution = softmax(counts_array_norm)

  # A good way to sample from a distribution is the following function from numpy
  x = np.random.multinomial(n = 1, pvals = final_distribution)
  selected_word = np.argmax(x).flatten()
  return selected_word[0]

sample_from([1,1,1,17], temperature = 0.5)


3

## TASK D

In [11]:
from pprint import pprint

def generate(ngrams, n, max_len = 40, temperature = 1.0, prompt = None):
  """
  ngrams: the master dict
  max_len: how many words at maximum
  temperature: the generation temperature
  prompt: the initial prompt, as a tuple, if not given n-1 <bos> symbols will be used
  """

  if prompt is None:
    prompt = ["<bos>"] * (n-1)

  generated = list(prompt) # This list will grow with the words inserted

  for _ in range(max_len):


    # CONTINUE HERE

    if generated[-1] == "<eos>": # Stop on end of sequence
      break

  return generated


for temp in (0.1,0.5,1.0,2.0,5.0):
    generated=generate(ngrams=ngrams,n=5,max_len=60,temperature=temp)
    print(f"Temp={temp}:")
    pprint(" ".join(generated))
    print("-----------")


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




('<bos>', '<bos>', '<bos>', '<bos>') {'I': 20083, 'If': 1464, 'This': 13830, 'Oh': 323, 'Whoever': 23, 'When': 1225, 'Who': 100, 'It': 2011, 'Never': 54, 'Its': 90, 'Today': 28, 'My': 737, 'Pros': 3, '1st': 81, 'There': 1108, 'En': 2, 'Without': 64, 'Amateur': 3, 'OK': 519, 'Some': 263, 'You': 456, 'THE': 161, 'A': 2732, 'Really': 72, 'Spoilers': 75, 'As': 1054, 'WARNING': 92, 'Jill': 2, 'Lifetime': 1, 'The': 6269, 'Ned': 3, 'They': 96, 'From': 244, 'Holy': 21, 'SWING': 1, 'Protocol': 2, 'What': 1143, 'Outlandish': 2, 'this': 505, 'Three': 71, 'Not': 398, 'Of': 125, 'How': 321, 'An': 455, 'or': 15, 'Devil': 5, 'Sexo': 1, 'Sometime': 5, 'Terrible': 49, 'Assuming': 1, 'Sometimes': 86, 'Nine': 6, 'Weak': 11, 'All': 183, '6': 2, 'Robert': 74, 'Story': 35, 'Are': 31, 'Ah': 75, 'Uninspired': 2, 'Having': 309, 'Saw': 141, 'Beyond': 18, 'William': 50, 'In': 1357, 'After': 825, 'Caught': 30, 'Was': 36, 'Andie': 3, 'SPOILER': 74, 'it': 72, 'LIGHT': 1, 'Considering': 51, 'Yes': 281, 'Closer': 1,